### X + Y Regression with Complex Numbers

This is another regression example using complex data, and modeling the surface X + Y = Z, where X and Y are both complex numbers

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math

linear = lambda x: x

def layer(act_func, input_val, input_num, output_num):
    W = tf.Variable(tf.random_normal([input_num, output_num], stddev=0.03), dtype=tf.float32)
    b = tf.Variable(tf.random_normal([output_num], stddev=0.03), dtype=tf.float32)
    layer = act_func(tf.matmul(input_val,W) + b)
    return layer

epochs = 15000
learning_rate = 0.001
num_points = 50
num_nodes = 32

#used for inputting into layers
input_tf = tf.placeholder(tf.float32, [None,4])

#actual data points
x_data = np.linspace(0, 20, num_points) + np.linspace(-10, 0, num_points)
y_data = np.linspace(-30, 10, num_points) + np.linspace(20, 30, num_points)
np.random.shuffle(x_data), np.random.shuffle(y_data)
x_data, y_data = np.meshgrid(x_data, y_data)

#prepping data to be input
x_train = x_data.reshape(-1,1)
y_train = y_data.reshape(-1,1)
input_data = np.hstack((x_train.real, x_train.imag, y_train.real, y_train.imag))

z = tf.complex(input_tf[:,0:1], input_tf[:,1:2]) + tf.complex(input_tf[:,2:3], input_tf[:,3:4])

#making the layers
layers = []
dicts={}
layers += [layer(tf.nn.leaky_relu, input_tf, 4, num_nodes)]
for i in range(3):
    dicts[i] = layer(tf.nn.leaky_relu, layers[-1], num_nodes, num_nodes)
    layers += [dicts[i]]

output = layer(linear, layers[-1], num_nodes, 2)

#prepping the loss functions
loss_real = tf.losses.mean_squared_error(tf.real(z), output[:,0:1])
loss_imag = tf.losses.mean_squared_error(tf.imag(z), output[:,1:2])

train = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_imag+loss_real)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(epochs+1):
        predict, _, curr_real, curr_imag = sess.run([output, train, loss_real, loss_imag], feed_dict={input_tf: input_data})
        if i % 1000 == 0:
            print("Epoch: %s \t Loss: %s" % (i, curr_real+curr_imag))
